# Recommender System - Flickr

This dataset is collected from online photo-sharing social media platform Flickr. People share photos with others on Flickr and make connections (friends). We will try to recommend a list of items (photos) to each user given rich information on the social media platform.

A recommender system is an algorithm that aims to predict a person’s preference for items or products in order to provide appropriate recommendations to them. When the user data available is limited to indirect observations of behaviour, this is referred to as implicit feedback. For our case, we have a dataset collected from the online photo-sharing platform Flickr, consisting of implicit feedback in the form of user-item interactions representing whether a user has ‘liked’ a particular photo. In addition, we have a dataset of social information, indicating which users are friends; and two feature representation datasets for users and items respectively. We will apply and compare three main recommender system algorithms for this task and evaluate their performances – content-based, collaborative filtering and latent factor-based matrix factorisation.

## Libraries

In [1]:
# !pip install implicit

In [341]:
import pandas as pd
import numpy as np
from itertools import product
import random
from math import log
import pickle
import time
import collections

import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sparse

In [2]:
# Read in data
train = pd.read_csv('flickr_train_data.csv')
validation = pd.read_csv('flickr_validation_data.csv')
test = pd.read_csv('flickr_test_data.csv')
item_fea = pd.read_csv('flickr_item_fea.csv')
item_fea.drop(item_fea.columns[0], axis=1, inplace=True)
user_fea = pd.read_csv('flickr_user_fea.csv')
user_fea.drop(user_fea.columns[0], axis=1, inplace=True)
all_links = pd.read_csv('flickr_links.csv')

In [3]:
# Preview data
train.head()

,user_id,item_id,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [4]:
# Preview number of photo interactions for most and least active users
train['user_id'].value_counts()

2909    691
1128    672
1664    612
295     558
1870    478
       ... 
3211      6
242       6
1777      6
1802      6
813       6
Name: user_id, Length: 3466, dtype: int64

In [5]:
# Preview number of user interactions for most and least interacted photos
train['item_id'].value_counts()

49      110
2486    101
3990     91
774      85
1429     79
       ... 
2511      4
2922      4
2540      4
829       4
3716      4
Name: item_id, Length: 9004, dtype: int64

In [6]:
train['rating'].value_counts()

1    110129
Name: rating, dtype: int64

All rating values are 1 because the data is implicit (a rating of 1 represents an interaction between the user and the item).

In [7]:
# Create sparse matrix of user-item interactions
sparse_item_user = sparse.csr_matrix((train['rating'].astype(float), (train['item_id'], train['user_id'])))
sparse_user_item = sparse.csr_matrix((train['rating'].astype(float), (train['user_id'], train['item_id'])))

The scores that each algorithm provides for user-item interaction predictions allowed for reverse-ordering with respect to a candidate list of items for each user, which could then be treated as ranks. The model is ultimately evaluated using the normalized discounted cumulative gain, which places importance on both good recommendations as well as the order, or rank, of those recommendations. In particular, higher rankings of relevant recommendations are emphasised by this evaluation method because they are discounted less before the values are accumulated.

We don’t have any data on the ranks of items for users, only interactions and non-interactions. For our validation set, we have 1 item interaction for each user and 99 non-interactions. As a proxy, to evaluate the models as they are being built, the target item for each user (rating of 1) is discounted by the log of the rank (base 2) assigned to it by the model, which is then averaged out across all users: 

$$\frac{\sum^{𝑈}_{𝑢=0} \frac{1}{log_{2}r_𝑢}}{|𝑈|}$$

$r_{𝑢}$ the model’s predicted rank of item with rating 1 for user 𝑢 <br>
$|𝑈|$ the cardinality of the set of all users

The value of 1 is used for a predicted rank of 1 to avoid a zero-division error and items that rank below 15 are assigned a zero value as ultimately, only the top 15 of 100 items for each user will be recommended. The average discounted gain should be maximised. The average rank of the interaction item across users, which should be a smaller value, is also calculated.

The `rank_items` function below, will provide recommendations based on scores fed into it and mask any user-item values where the user has already interacted with that item.

In [8]:
def rank_items(sparse_user_item, user, candidate_idx, scores, num_items):
    '''For a given user, list of items and scores; remove any items that a user has already interacted with, 
    then sort from highest to lowest and return the items and scores.
    '''
    
    # Specify subset of items
    items = candidate_idx
    
    # Masks items that have already been interacted with so they are not recommended.
    # Get the interactions scores from the sparse user item matrix
    user_interactions = sparse_user_item[user,items].toarray()
    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    user_interactions = user_interactions.reshape(-1) + 1
    # Make articles already interacted zero
    user_interactions[user_interactions > 1] = 0
    
    # Rescale recommendation scores
    min_max = MinMaxScaler()
    scores_scaled = min_max.fit_transform(scores.reshape(-1,1))[:,0]
    # Item already interacted have their recommendation multiplied by zero
    recommend_vector = user_interactions * scores_scaled
    # Sort the indices of the item into order of best recommendations
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    # Start empty list to store titles and scores
    items_list = []
    scores_list = []
    
    for idx in item_idx:
        # Append item index and scores to the list
        items_list.append(items[idx])
        scores_list.append(recommend_vector[idx])

    recommendations = items_list, scores_list
    
    return recommendations

In [145]:
def get_ranks_and_dg(users, items, scores, ranks, test_set):
    '''Get average rank and discount cumulative gain of interacted item from a test set'''
    
    # Get rankings of the 1 item interaction for each user from the validation set
    recommendations_df = pd.DataFrame(list(zip(users, items, scores, ranks)), 
                                      columns =['user_id', 'item_id', 'scores', 'ranks'])

    # Display the average ranking (1-100) of each user's 'correct' item interaction from validation set
    merged_df = recommendations_df.merge(test_set[test_set.rating==1], on='user_id')
    merged_df = merged_df[merged_df.item_id_x==merged_df.item_id_y]
    merged_df['dg'] = merged_df['ranks'].apply(lambda x: (1/log(x,2) if x > 1 else x) if x <= 15 else 0)
    avg_rank = round(merged_df.ranks.sum() / sparse_user_item.shape[0], 4)
    avg_dg = round(merged_df.dg.sum() / sparse_user_item.shape[0], 5)
    
    return avg_dg, avg_rank

## Content-based recommender

Content-based recommender systems are aimed at utilising the preferences of users and the features of items (the content) in order to recommend items that closely align to the user’s preferences. With a vectorized set of item features and a corresponding vectorized user profile, the Cosine similarity of the user U and the item I can be calculated as follows:

$$cos(U,I)=\frac{\sum^𝑁_𝑖{U_𝑖 I_𝑖}}{\sqrt{\sum^N_{i=1}{U^2_𝑖}}\sqrt{\sum^N_{i=1}{I^2_𝑖}}}$$

The Cosine similarity can be calculated between all users and items and for a given user, the items with the highest similarity can be recommended to them.

Here, the user and item feature representation files are used to build a content-based recommender.

In [9]:
# Matrix of all user-item cosine similarities based on user and item features
content_similarity = cosine_similarity(user_fea.to_numpy(), item_fea.to_numpy())

In [10]:
def cb_recommend(user_id, sparse_user_item, test_set, num_items=15):
    '''Given a set of possible items, this function uses the content-based approach, calculating cosine similarities
    between users and items based on user and item feature sets, then returns an ordered ranking.
    '''
    
    # Get item IDs from test set that we need to calculate scores for
    candidate_idx = test_set[test_set.user_id==user_id].item_id.to_list()
    
    # Get relevant cosine similarities
    scores = content_similarity[user_id][candidate_idx]

    return rank_items(sparse_user_item, user_id, candidate_idx, scores, num_items)

Below, the function is run on the validation set for each user and the results are compiled in a dataframe. Each user has one item that should be recommended and 99 items that shouldn't be recommended. Therefore, we are interested in the correct item being ranked as high as possible for each user. The average rank of the item that each user should be recommended is displayed below. It is not a very good result, given that the average rank ranges from 1 to 100, so 50.5 would be equivalent to random assignment.

In [144]:
# Build lists of recommendations
users, items, scores = [], [], []
ranks = [x for x in range(1,101)] * sparse_user_item.shape[0]
for i in range(sparse_user_item.shape[0]):
    users += [i] * 100
    recommendations = cb_recommend(i, sparse_user_item, validation, num_items=100)
    items += recommendations[0]
    scores += recommendations[1]

# Get average discount gain and average rank
avg_dg, avg_rank = get_ranks_and_dg(users, items, scores, ranks, validation)

# Display the average ranking (0-99) of each user's 'correct' item interaction from validation set
print(f'    Average rank: {avg_rank}\n\
    Average discount gain: {avg_dg}\n')

    Average rank: 46.4986
    Average discount gain: 0.1148



## Memory-Based Collaborative Filtering

Memory-based collaborative filtering uses information on past ratings or interactions of users and items. It looks at the similarities of a particular item with other items (item-based) or a particular user with other users (user-based) and then uses the ratings/interactions of the similar items or from the similar users to predict a rating or score for the particular user-item pair.

### Item-based

The item-based approach to collaborative filtering finds the similarity between an item $i$ and the $N$ most similar items and then predicts a rating (or in our case, a score as there are no ratings, only interactions) to prioritise recommendations. The formula below will be applied.

$$r_{u,i} = \bar{r}_i + \frac{\sum_{j\in N_i}sim(i,j)(r_{u,j}-\bar{r}_j)}{\sum_{j\in N_i}sim(i,j)}, \text {where:}$$ <br>
<center>$\bar{r}_i$ represents the proportion of all users that have interacted with item $i$</center>
<center>$sim(i,j)$ is the similarity score between items $i$ and $j$ with $j$ being one of the $N$ most similar items to $i$</center>
<center>$\bar{r}_j$ represents the proportion of all users that have interacted with item $j$</center>
<center>$r_{u,j}$ is 1 if the user $u$ has interacted with similar item $j$ and 0 otherwise</center>

The function below will calculate this user-item score:

In [167]:
def user_item_rating(r_bar_item, item_item_sim, r_bar_sim_items, r_user_sim_items, n):
    '''Calculates user-item score (item-based CF)'''
    
    numerator = 0
    
    # Calculate numerator of predicted rating formula (n is neighbour)
    for x in range(n): 
        numerator += item_item_sim[x]*(r_user_sim_items[x] - r_bar_sim_items[x])
    return r_bar_item + numerator / sum(item_item_sim)

For $sim(i,j)$, we can use cosine similarity or jaccard similarity. Jaccard similarity score is commonly used with implicit data and is calculated as follows:

$$sim(i,j)=\frac{|A\cap B|}{|A\cup B|}$$<br>
where $A$ is the set of users that have interacted with item $i$ and $B$ is the set of users that have interacted with item $j$

The function below, taken from [this](https://stackoverflow.com/questions/32805916/compute-jaccard-distances-on-sparse-matrix) stack overflow post, implements a jaccard similarity matrix:

In [149]:
def pairwise_jaccard(X):
    """Computes the Jaccard distance between the rows of `X`."""
    
    X = X.astype(bool).astype(int)

    intrsct = X.dot(X.T) # Numerator
    row_sums = intrsct.diagonal()
    unions = row_sums[:,None] + row_sums - intrsct # Denominator
    return np.asarray(intrsct / unions)

The function below create the recommendations. It is similar to the content-based recommender but the calculation of the recommendation vector has changed.

In [169]:
def ibcf_recommend(user_id, sparse_user_item, test_set, item_avg_interactions, item_n_most_similar_idx, 
                   item_n_most_similar, neighbours, num_items=15):
    '''For a user, apply item-based collaborative filtering to calculate scores for candidiate items and order them from
    highest to lowest as rank order.
    '''
    
    # Get list of item (IDs) that the user has interacted with
    user_items = train[train.user_id==user_id].item_id.tolist()
    
    # Get item IDs from test set that we need to calculate scores for
    candidate_idx = test_set[test_set.user_id==user_id].item_id.to_list()
    
    # Get scores for each candidate item
    u = user_id
    rating_score = []
    for i in candidate_idx:
        # Get values for item-based collaborative filtering ratings calculations
        r_bar_item = item_avg_interactions[i]
        r_bar_sim_items = item_avg_interactions[[int(x) for x in item_n_most_similar_idx[i]]]
        r_user_sim_items = sparse_user_item[u, item_n_most_similar_idx[i]].toarray().flatten()
        item_item_sim = item_n_most_similar[i]
        
        # Compute rating score of user u and item i and append
        rating_score.append(user_item_rating(r_bar_item, item_item_sim, r_bar_sim_items, r_user_sim_items, neighbours))
    
    # Get recommendation scores
    scores = np.array(rating_score)

    return rank_items(sparse_user_item, user_id, candidate_idx, scores, num_items)

Get components for `user_item_rating` calculation:

In [161]:
def cf_components(similarity_measure, sparse_matrix):
    
    if s == 'Cosine':
        similarity_matrix = cosine_similarity(sparse_matrix, sparse_matrix)
    elif s == 'Jaccard':
        similarity_matrix = pairwise_jaccard(sparse_matrix)

    # Create copies to modify
    item_n_most_similar = similarity_matrix.copy()
    item_n_most_similar_idx = similarity_matrix.copy()

    # Re-order rows in similarities so that each row is ordered from most to least similar
    for row in range(len(item_n_most_similar)):
        # Sort each row from highest to lowest to get the top similarities and their indices
        item_n_most_similar[row] = np.sort(similarity_matrix[row])[::-1]
        item_n_most_similar_idx[row] = np.argsort(similarity_matrix[row])[::-1]

    # Keep only first n columns to use as neighbours
    item_n_most_similar = item_n_most_similar[:,1:n+1]
    item_n_most_similar_idx = item_n_most_similar_idx[:,1:n+1]      

    # Get proportion of each items' number of user interactions divided by total number of users
    item_avg_interactions = sparse_matrix.sum(axis=1)/sparse_matrix.shape[1]
    item_avg_interactions = np.squeeze(np.asarray(item_avg_interactions))

    return item_avg_interactions, item_n_most_similar_idx, item_n_most_similar

Below we tune the hyperparameter for the number of neighbours to use and which similarity measure:

In [170]:
for n in range(100, 1001, 100):
    
    for s in ['Cosine', 'Jaccard']:
        
        # Get collaborative formula components
        item_avg_interactions, item_n_most_similar_idx, item_n_most_similar = cf_components(s, sparse_item_user)
        
        # Build lists of recommendations
        users, items, scores = [], [], []
        ranks = [x for x in range(1,101)] * sparse_user_item.shape[0]
        num_items = 100
        for i in range(sparse_user_item.shape[0]):
            users += [i] * num_items
            recommendations = ibcf_recommend(i, sparse_user_item, validation, item_avg_interactions,
                                       item_n_most_similar_idx, item_n_most_similar, n, num_items)
            items += recommendations[0]
            scores += recommendations[1]

        # Get average discount gain and average rank
        avg_dg, avg_rank = get_ranks_and_dg(users, items, scores, ranks, validation)
        
        # Display
        print(f'\n\
        Neighbours: {n}\n\
        Similarity: {s}\n\
        Average rank: {avg_rank}\n\
        Average DG: {avg_dg}\n')


        Neighbours: 100
        Similarity: Cosine
        Average rank: 39.5499
        Average DG: 0.17145


        Neighbours: 100
        Similarity: Jaccard
        Average rank: 39.8266
        Average DG: 0.17351


        Neighbours: 200
        Similarity: Cosine
        Average rank: 38.0675
        Average DG: 0.18495


        Neighbours: 200
        Similarity: Jaccard
        Average rank: 38.1757
        Average DG: 0.1881


        Neighbours: 300
        Similarity: Cosine
        Average rank: 37.0903
        Average DG: 0.19075


        Neighbours: 300
        Similarity: Jaccard
        Average rank: 37.2646
        Average DG: 0.19136


        Neighbours: 400
        Similarity: Cosine
        Average rank: 36.7764
        Average DG: 0.19457


        Neighbours: 400
        Similarity: Jaccard
        Average rank: 36.8122
        Average DG: 0.19691


        Neighbours: 500
        Similarity: Cosine
        Average rank: 36.6714
        Average DG: 0.19735

### User-based

Similarly, the user-based approach can be used. Instead of considering the similarity of the item with the most similar items, it considers the similarity of the user with the $N$ most similar users. The formula is as follows:

$$r_{u,i} = \bar{r}_u + \frac{\sum_{v\in N_u}sim(u,v)(r_{v,i}-\bar{r}_v)}{\sum_{v\in N_u}sim(u,v)}, \text {where:}$$ <br>
<center>$\bar{r}_u$ represents the proportion of all items that user $u$ has interacted with</center>
<center>$sim(u,v)$ is the similarity score between users $u$ and $v$ with $v$ being one of the $N$ most similar users to $u$</center>
<center>$\bar{r}_v$ represents the proportion of all items that user $v$ has interacted with</center>
<center>$r_{v,i}$ is 1 if the item $i$ has been interacted with by similar user $v$ and 0 otherwise</center>
</font>

The same logic is applied for the user-based approach:

In [172]:
def item_user_rating(r_bar_user, user_user_sim, r_bar_sim_users, r_item_sim_users, neighbours):
    '''Calculates item-user score (user-based CF)'''
    
    numerator = 0
    
    # Calculate numerator of predicted rating formula (n is neighbour)
    for x in range(neighbours): 
        numerator += user_user_sim[x]*(r_item_sim_users[x] - r_bar_sim_users[x])
    return r_bar_user + numerator / sum(user_user_sim)

Modify the item-based collaborative filter recommender algorithm to a user-based collaborative filter approach:

In [173]:
def ubcf_recommend(user_id, sparse_user_item, sparse_item_user, test_set, user_avg_interactions,
                   user_n_most_similar_idx, user_n_most_similar, neighbours, num_items=15):
    '''For a user, apply user-based collaborative filtering to calculate scores for candidiate items and order them from
    highest to lowest as rank order.
    '''

    # Get list of item (ids) that user has interacted with
    user_items = train[train.user_id==user_id].item_id.tolist()
    
    # Get item IDs from test set that we need to calculate scores for
    candidate_idx = test_set[test_set.user_id==user_id].item_id.to_list()
    
    u = user_id
    rating_score = []

    # Get values for user-based collaborative filtering ratings calculations
    r_bar_sim_users = user_avg_interactions[[int(x) for x in user_n_most_similar_idx[u]]]
    r_bar_user = user_avg_interactions[u]
    user_user_sim = user_n_most_similar[u]

    for i in candidate_idx:
        # Get values for user-based collaborative filtering ratings calculations
        r_item_sim_users = sparse_item_user[i, user_n_most_similar_idx[u]].toarray().flatten()

        # Compute rating score of item u and user i and append
        rating_score.append(item_user_rating(r_bar_user, user_user_sim, r_bar_sim_users, r_item_sim_users, neighbours))
    
    # Get recommendation scores
    scores_vec = np.array(rating_score)

    return rank_items(sparse_user_item, user_id, candidate_idx, scores_vec, num_items)

Try different neighbourhood sizes of similar users:

In [176]:
# Tune the number of neighbours
for n in range(100, 1001, 100):

    for s in ['Cosine', 'Jaccard']:
        
        # Get collaborative formula components
        user_avg_interactions, user_n_most_similar_idx, user_n_most_similar = cf_components(s, sparse_user_item)

        # Build lists of recommendations
        users, items, scores = [], [], []
        ranks = [x for x in range(1,101)] * sparse_user_item.shape[0]
        num_items = 100
        for j in range(sparse_user_item.shape[0]):
            users += [j] * num_items
            recommendations = ubcf_recommend(j, sparse_user_item, sparse_item_user, validation, user_avg_interactions,
                                             user_n_most_similar_idx, user_n_most_similar, n, num_items)
            items += recommendations[0]
            scores += recommendations[1]

        # Get average discount gain and average rank
        avg_dg, avg_rank = get_ranks_and_dg(users, items, scores, ranks, validation)
        
        # Display
        print(f'\n\
        Neighbours: {n}\n\
        Similarity: {s}\n\
        Average rank: {avg_rank}\n\
        Average DG: {avg_dg}\n')


        Neighbours: 100
        Similarity: Cosine
        Average rank: 56.2614
        Average DG: 0.17452


        Neighbours: 100
        Similarity: Jaccard
        Average rank: 57.0828
        Average DG: 0.17255


        Neighbours: 200
        Similarity: Cosine
        Average rank: 42.5427
        Average DG: 0.18988


        Neighbours: 200
        Similarity: Jaccard
        Average rank: 42.4836
        Average DG: 0.19272


        Neighbours: 300
        Similarity: Cosine
        Average rank: 39.9048
        Average DG: 0.19293


        Neighbours: 300
        Similarity: Jaccard
        Average rank: 39.5424
        Average DG: 0.19486


        Neighbours: 400
        Similarity: Cosine
        Average rank: 39.1431
        Average DG: 0.19469


        Neighbours: 400
        Similarity: Jaccard
        Average rank: 38.8604
        Average DG: 0.19593


        Neighbours: 500
        Similarity: Cosine
        Average rank: 39.1018
        Average DG: 0.1930

## Matrix Factorisation

Matrix factorisation is a way of decomposing the large sparse user-item interactions matrix into a product of two smaller latent matrices, a user matrix and an item matrix, that can most closely reconstruct the original known values. The latent matrices share a dimension length **k**, being the number of factors specified. The objective is to minimise the sum of squared errors of the known ratings minus the equivalent predicted values in the matrix reconstructed from the two latent matrices. A regularisation term **𝝀** can also be added to the objective function to prevent overfitting from selecting too many factors. 

$$𝑎𝑟𝑔𝑚𝑖𝑛\sum_{(𝑢,𝑖)∈𝑆}(𝑅_{𝑢𝑖}-𝑝_𝑢𝑞^𝑇_𝑖)^2+𝜆(‖𝑝_𝑢‖^2+‖𝑞_𝑖‖^2)$$
$$𝑆=(𝑢,𝑖)\text{ }\big{|}\text{ }𝑅_{𝑢𝑖} \text{𝑖𝑠 𝑘𝑛𝑜𝑤𝑛}$$

$𝑅_{𝑢𝑖}$ the rating of item 𝒊 by user 𝑢 <br>
$𝑝_𝑢$ the vector for user 𝑢 from the latent user matrix <br>
$𝑞_𝑖$ the vector for item 𝒊 from the latent item matrix

### Alternating Least Squares (ALS)

Alternating Least Squares (ALS) is a popular approach where the solving of the objective function is broken down into two steps. For each step, one of the latent matrices is fixed and the other is optimised for each user-item vector combination. An optimiser such as stochastic gradient descent can be used and the process is iteratively applied. The following function takes hyperparameters, factorises a matrix using ALS and returns the factors.

In [177]:
def get_factorised_vecs(sparse_user_item, alpha, factors, reg, iterations):
    '''Train an ALS model and return the smaller factorised matrices'''
        
    # Adjust for rate of confidence of interactions
    data = (sparse_user_item * alpha).astype('double')
    
    # Fit the model
    model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=reg, iterations=iterations, random_state=0)
    model.fit(data)

    # Get the trained user and item vectors, converted to csr matrices
    user_vecs = sparse.csr_matrix(model.user_factors)
    item_vecs = sparse.csr_matrix(model.item_factors)
    
    return user_vecs, item_vecs

In [19]:
def als_recommend(user_id, sparse_user_item, user_vecs, item_vecs, test_set, num_items=15):
    '''For a user, a list of candidate items and user & item matrix factors, calculate item scores, rank them and return
    ranked list of items and scaled scores.
    '''
    
    # Get item IDs from test set that we need to calculate scores for
    candidate_idx = test_set[test_set.user_id==user_id].item_id.to_list()

    # Dot product of user vector and item vectors to provide a score for values of original matrix
    scores_vec = user_vecs[user_id,:].dot(item_vecs.T[:,candidate_idx]).toarray()

    return rank_items(sparse_user_item, user_id, candidate_idx, scores_vec, num_items)

In [128]:
# Create a function for tuning the hyperparameters
def tune_als(alpha=1, factors=100, reg=0.01, iterations=50, num_items=100):
    '''Train ALS model, make recommendations and evaluate'''
    
    # Get the ALS matrix factors used to build recommendations
    user_vecs, item_vecs = get_factorised_vecs(sparse_user_item, alpha, factors, reg, iterations)

    # Build lists of recommendations
    users, items, scores = [], [], []
    ranks = [x for x in range(1,101)] * sparse_user_item.shape[0]
    for i in range(sparse_user_item.shape[0]):
        users += [i] * num_items
        recommendations = als_recommend(i, sparse_user_item, user_vecs, item_vecs, validation, num_items)
        items += recommendations[0]
        scores += recommendations[1]

    # Get average rank and average discount gain
    avg_dg, avg_rank = get_ranks_and_dg(users, items, scores, ranks, validation)
    
    # Display
    print(f'    Alpha: {alpha} \n\
    Factors: {factors} \n\
    Regularisation: {reg} \n\
    Iterations: {iterations} \n\
    Average rank: {avg_rank} \n\
    Average DG: {avg_dg}\n')
    return avg_dg, avg_rank, alpha, factors, reg

In [129]:
# Tune hyperparameters for ALS model
best_avg_dg = 0
for f in range(2,21):
    for a in range(5,101,5):
        results = tune_als(alpha=a, factors=f)
        if results[0] > best_avg_dg:
            best_avg_dg = results[0]
            settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 5 
    Factors: 2 
    Regularisation: 0.01 
    Iterations: 50 
    Average rank: 38.4838 
    Average DCG: 0.14867



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 10 
    Factors: 2 
    Regularisation: 0.01 
    Iterations: 50 
    Average rank: 36.8231 
    Average DCG: 0.16244



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 90 
    Factors: 20 
    Regularisation: 0.01 
    Iterations: 50 
    Average rank: 31.9437 
    Average DCG: 0.20805



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 95 
    Factors: 20 
    Regularisation: 0.01 
    Iterations: 50 
    Average rank: 32.0268 
    Average DCG: 0.20667



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 100 
    Factors: 20 
    Regularisation: 0.01 
    Iterations: 50 
    Average rank: 32.092 
    Average DCG: 0.20753



In [130]:
settings

(0.25164, 27.2135, 35, 9, 0.01)

The optimal local values for alpha and number of factors, with regularisation fixed are 35 and 9 respectively. Now to find a good regularisation coefficient.

In [132]:
# Tune hyperparameters for ALS model
best_avg_dg = settings[0]
for f, a, r in product(range(7,11), range(30,81,5), [10*i for i in range(1,16)]):
    results = tune_als(alpha=a, factors=f, reg=r)
    if results[0] > best_avg_dg:
        best_avg_dg = results[0]
        settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 30 
    Factors: 7 
    Regularisation: 10 
    Iterations: 50 
    Average rank: 27.1673 
    Average DCG: 0.24651



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 30 
    Factors: 7 
    Regularisation: 20 
    Iterations: 50 
    Average rank: 27.1789 
    Average DCG: 0.24624



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 80 
    Factors: 10 
    Regularisation: 130 
    Iterations: 50 
    Average rank: 27.1728 
    Average DCG: 0.24436



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 80 
    Factors: 10 
    Regularisation: 140 
    Iterations: 50 
    Average rank: 27.1154 
    Average DCG: 0.24463



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 80 
    Factors: 10 
    Regularisation: 150 
    Iterations: 50 
    Average rank: 27.0314 
    Average DCG: 0.24623



In [135]:
settings

(0.25363, 26.7853, 45, 9, 140)

### Modifications: ALS + link data score adjustment

The ALS model performs the best so far. Below, social link data is also incorporated to adjust the scores if a social link has interacted with one of the items that is to be ranked for the user.

In [178]:
def als_recommend_2(user_id, sparse_user_item, user_vecs, item_vecs, test_set, num_items=15, link_multiply=1.1):
    '''For a user, a list of candidate items and user & item matrix factors, calculate item scores, rank them and return
    ranked list of items and scaled scores. Includes adjustments to score for item interactions of social links of the user.
    '''
    
    # Get item IDs from test set that we need to calculate scores for
    candidate_idx = test_set[test_set.user_id==user_id].item_id.to_list()

    # Dot product of user vector and item vectors to provide a score for values of original matrix
    scores = user_vecs[user_id,:].dot(item_vecs.T[:,candidate_idx]).toarray()
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    scores_scaled = min_max.fit_transform(scores.reshape(-1,1))[:,0]
    
    # Select users followed by user_id
    links = all_links[all_links.src==user_id].des.to_list()
    links += all_links[all_links.des==user_id].src.to_list() # bi-directional links
    # Select all items interacted with by followed users
    link_items = train[train.user_id.isin(links)].item_id
    # Filter link items to those in the candidate items for the user
    link_items_filtered = link_items[link_items.isin(candidate_idx)].to_list()

    # Increase scores of followed users items if also on the candidate items list for user
    for i in link_items_filtered:
        scores_scaled[candidate_idx.index(i)] = scores_scaled[candidate_idx.index(i)]*link_multiply
    
    return rank_items(sparse_user_item, user_id, candidate_idx, scores_scaled, num_items)

In [184]:
# Create a function for tuning the hyperparameters
def tune_als_2(alpha=1, factors=100, reg=0.01, iterations=50, num_items=100, link_multiply=1.1):
    '''Train ALS model with score adjustments for social links, make recommendations and evaluate'''
        
    # Get the ALS matrix factors used to build recommendations
    user_vecs, item_vecs = get_factorised_vecs(sparse_user_item, alpha, factors, reg, iterations)

    # Build lists of recommendations
    users, items, scores = [], [], []
    ranks = [x for x in range(1,101)] * sparse_user_item.shape[0]
    for i in range(sparse_user_item.shape[0]):
        users += [i] * num_items
        recommendations = als_recommend_2(i, sparse_user_item, user_vecs, item_vecs, validation, num_items, link_multiply)
        items += recommendations[0]
        scores += recommendations[1]

    # Get average rank and average discount gain
    avg_dg, avg_rank = get_ranks_and_dg(users, items, scores, ranks, validation)
    
    print(f'    Alpha: {alpha} \n\
    Factors: {factors} \n\
    Regularisation: {reg} \n\
    Link multiplier: {link_multiply} \n\
    Iterations: {iterations} \n\
    Average rank: {avg_rank} \n\
    Average DG: {avg_dg}\n')
    return avg_dg, avg_rank, alpha, factors, reg, link_multiply

In [259]:
# Tune hyperparameters for ALS model with link score adjustment
best_avg_dg = settings[0]
for f, a, r, l in product(range(7,11), range(40,81,10), range(120,161,10), [1+i/20 for i in range(5)]):
    results = tune_als_2(alpha=a, factors=f, reg=r, link_multiply=l)
    if results[0] > best_avg_dg:
        best_avg_dg = results[0]
        settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 40 
    Factors: 7 
    Regularisation: 120 
    Link multiplier: 1.0 
    Iterations: 50 
    Average rank: 27.0987 
    Average DG: 0.24627



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 40 
    Factors: 7 
    Regularisation: 120 
    Link multiplier: 1.05 
    Iterations: 50 
    Average rank: 26.9991 
    Average DG: 0.24956



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 80 
    Factors: 10 
    Regularisation: 160 
    Link multiplier: 1.1 
    Iterations: 50 
    Average rank: 26.796 
    Average DG: 0.25632



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 80 
    Factors: 10 
    Regularisation: 160 
    Link multiplier: 1.15 
    Iterations: 50 
    Average rank: 26.7902 
    Average DG: 0.25496



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 80 
    Factors: 10 
    Regularisation: 160 
    Link multiplier: 1.2 
    Iterations: 50 
    Average rank: 26.8099 
    Average DG: 0.25276



In [190]:
settings

(0.25813, 26.6293, 70, 9, 130, 1.1)

### Modifications: ALS + link data & content-based score adjustment

One last adjustment. In addition to adding link adjustments, we will also try altering scores of the re-created ALS matrix by using some fraction of the scores fron the content-based recommender approach and see if it offers up any improvement.

In [32]:
def als_recommend_3(user_id, sparse_user_item, user_vecs, item_vecs, test_set, 
                    num_items=15, link_multiply=1.1, content_adj=0.1):
    '''For a user, a list of candidate items and user & item matrix factors, calculate item scores, rank them and return
    ranked list of items and scaled scores. Includes adjustments to score for item interactions of social links of the user
    and from a content-based recommender score.
    '''
    
    # Get item IDs from test set that we need to calculate scores for
    candidate_idx = test_set[test_set.user_id==user_id].item_id.to_list()

    # Get dot product of user vector and all item vectors
    scores = user_vecs[user_id,:].dot(item_vecs.T[:,candidate_idx]).toarray()
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    scores_scaled = min_max.fit_transform(scores.reshape(-1,1))[:,0]
    
    # Get relevant cosine similarities
    cb_scores = content_similarity[user_id][candidate_idx]
    cb_scores_scaled = min_max.fit_transform(cb_scores.reshape(-1,1))[:,0]
  
    # Use content-based recommender scores to alter ALS scores 
    scores_scaled += cb_scores_scaled*content_adj
    # Rescale
    scores_scaled = min_max.fit_transform(scores_scaled.reshape(-1,1))[:,0]
    
    # Select users followed by selected user
    links = all_links[all_links.src==user_id].des.to_list()
    links += all_links[all_links.des==user_id].src.to_list() # bi-directional
    # Select all items interacted with by followed users
    link_items = train[train.user_id.isin(links)].item_id
    # Filter link items to those in the candidate items for the user
    link_items_filtered = link_items[link_items.isin(candidate_idx)].to_list()
    
    # Increase scores of followed users items if also on the candidate items list for user
    for i in link_items_filtered:
        scores_scaled[candidate_idx.index(i)] = scores_scaled[candidate_idx.index(i)]*link_multiply

    return rank_items(sparse_user_item, user_id, candidate_idx, scores_scaled, num_items)

In [203]:
# Function for tuning the hyperparameters
def tune_als_3(alpha=1, factors=100, reg=0.01, iterations=50, num_items=100, link_multiply=1.1, content_adj=0.1):
    '''Train ALS model with score adjustments for social links, make recommendations and evaluate'''
    
    # Get the ALS matrix factors used to build recommendations
    user_vecs, item_vecs = get_factorised_vecs(sparse_user_item, alpha, factors, reg, iterations)
    
    # Build lists of recommendations
    users, items, scores = [], [], []
    ranks = [x for x in range(1,101)] * sparse_user_item.shape[0]
    for i in range(sparse_user_item.shape[0]):
        users += [i] * num_items
        recommendations = als_recommend_3(i, sparse_user_item, user_vecs, item_vecs,
                                          validation, num_items, link_multiply, content_adj)
        items += recommendations[0]
        scores += recommendations[1]

    # Get average rank and average discount gain
    avg_dg, avg_rank = get_ranks_and_dg(users, items, scores, ranks, validation)
        
    print(f'    Alpha: {alpha} \n\
    Factors: {factors} \n\
    Regularisation: {reg} \n\
    Link multiplier: {link_multiply} \n\
    Content adjustment: {content_adj} \n\
    Iterations: {iterations} \n\
    Average rank: {avg_rank}\n\
    Average dg: {avg_dg}\n')
    return avg_dg, avg_rank, alpha, factors, reg, link_multiply, content_adj

In [263]:
# Tune hyperparameters for ALS model with link score adjustment and content-based adjustment
best_avg_dg = settings[0]
for f, a, r, c in product(range(8,11), range(70,86,5), range(130,151,5), [i/20 for i in range(5)]):
    results = tune_als_3(alpha=a, factors=f, reg=r, link_multiply=1.1, content_adj=c)
    if results[0] > best_avg_dg:
        best_avg_dg = results[0]
        settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 70 
    Factors: 8 
    Regularisation: 130 
    Link multiplier: 1.1 
    Content adjustment: 0.0 
    Iterations: 50 
    Average rank: 26.3381
    Average dg: 0.25491



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 70 
    Factors: 8 
    Regularisation: 130 
    Link multiplier: 1.1 
    Content adjustment: 0.05 
    Iterations: 50 
    Average rank: 26.2781
    Average dg: 0.25507



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 85 
    Factors: 10 
    Regularisation: 150 
    Link multiplier: 1.1 
    Content adjustment: 0.1 
    Iterations: 50 
    Average rank: 26.8731
    Average dg: 0.25221



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 85 
    Factors: 10 
    Regularisation: 150 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.8935
    Average dg: 0.25263



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 85 
    Factors: 10 
    Regularisation: 150 
    Link multiplier: 1.1 
    Content adjustment: 0.2 
    Iterations: 50 
    Average rank: 26.944
    Average dg: 0.25331



In [212]:
settings

(0.25945, 26.5167, 70, 9, 145, 1.1, 0.15)

We'll also try to find a model with a higher number of factors, we will need to include larger regularisation factors to avoid overfitting the training data.

In [249]:
# Tune hyperparameters
best_avg_dg = settings[0]
for f, a, r in product(range(50,201,25), [50,100,500,1000,5000,10000], [100,500,1000,5000,10000]):
    results = tune_als_3(alpha=a, factors=f, reg=r, link_multiply=1.1, content_adj=0.15)
    if results[0] > best_avg_dg:
        best_avg_dg = results[0]
        settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 50 
    Factors: 50 
    Regularisation: 100 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 32.2934
    Average dg: 0.21587



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 50 
    Factors: 50 
    Regularisation: 500 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 32.9397
    Average dg: 0.20465



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 10000 
    Factors: 200 
    Regularisation: 1000 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 29.9887
    Average dg: 0.22277



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 10000 
    Factors: 200 
    Regularisation: 5000 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 28.2121
    Average dg: 0.22577



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 10000 
    Factors: 200 
    Regularisation: 10000 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 34.5326
    Average dg: 0.19364



In [254]:
settings

(0.26554, 26.3203, 100, 150, 500, 1.1, 0.15)

The next few code cells are more runs of finetuning the model.

In [255]:
# Tune hyperparameters
best_avg_dg = settings[0]
for f, a, r in product(range(100,201,25), range(75,126,25), range(400, 601, 50)):
    results = tune_als_3(alpha=a, factors=f, reg=r, link_multiply=1.1, content_adj=0.15)
    if results[0] > best_avg_dg:
        best_avg_dg = results[0]
        settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 75 
    Factors: 100 
    Regularisation: 400 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.6408
    Average dg: 0.26131



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 75 
    Factors: 100 
    Regularisation: 450 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.6979
    Average dg: 0.26201



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 125 
    Factors: 200 
    Regularisation: 500 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.6541
    Average dg: 0.26348



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 125 
    Factors: 200 
    Regularisation: 550 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.2914
    Average dg: 0.26486



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 125 
    Factors: 200 
    Regularisation: 600 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.142
    Average dg: 0.26701



In [256]:
settings

(0.26701, 26.142, 125, 200, 600, 1.1, 0.15)

In [257]:
# Tune hyperparameters
best_avg_dg = settings[0]
for f, a, r in product(range(100,201,25), range(125,201,25), range(600, 1001, 50)):
    results = tune_als_3(alpha=a, factors=f, reg=r, link_multiply=1.1, content_adj=0.15)
    if results[0] > best_avg_dg:
        best_avg_dg = results[0]
        settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 125 
    Factors: 100 
    Regularisation: 600 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.1301
    Average dg: 0.26621



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 125 
    Factors: 100 
    Regularisation: 650 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.2452
    Average dg: 0.26134



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 200 
    Factors: 200 
    Regularisation: 900 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 25.9931
    Average dg: 0.26409



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 200 
    Factors: 200 
    Regularisation: 950 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.176
    Average dg: 0.26243



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 200 
    Factors: 200 
    Regularisation: 1000 
    Link multiplier: 1.1 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.5759
    Average dg: 0.25681



In [266]:
settings

(0.26711, 26.0632, 175, 175, 700, 1.1, 0.15)

In [297]:
# Tune hyperparameters
best_avg_dg = settings[0]
for f, a, r, l, c in product(range(100,176,25), range(100,121,5), range(475, 551, 25),
                      [1.08, 1.10, 1.12], [0.125, 0.15, 0.175]):
    results = tune_als_3(alpha=a, factors=f, reg=r, link_multiply=l, content_adj=c)
    if results[0] > best_avg_dg:
        best_avg_dg = results[0]
        settings = results

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 100 
    Factors: 100 
    Regularisation: 475 
    Link multiplier: 1.08 
    Content adjustment: 0.125 
    Iterations: 50 
    Average rank: 26.4611
    Average dg: 0.2653



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 100 
    Factors: 100 
    Regularisation: 475 
    Link multiplier: 1.08 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.472
    Average dg: 0.26543



  0%|          | 0/50 [00:00<?, ?it/s]

...

    Alpha: 120 
    Factors: 175 
    Regularisation: 550 
    Link multiplier: 1.12 
    Content adjustment: 0.125 
    Iterations: 50 
    Average rank: 26.2418
    Average dg: 0.26509



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 120 
    Factors: 175 
    Regularisation: 550 
    Link multiplier: 1.12 
    Content adjustment: 0.15 
    Iterations: 50 
    Average rank: 26.2429
    Average dg: 0.26604



  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 120 
    Factors: 175 
    Regularisation: 550 
    Link multiplier: 1.12 
    Content adjustment: 0.175 
    Iterations: 50 
    Average rank: 26.2398
    Average dg: 0.26601



In [298]:
settings

(0.26765, 26.2383, 115, 150, 550, 1.1, 0.175)

In [311]:
# Try last top model with higher content adjustment percentage
tune_als_3(alpha=115, factors=150, reg=550, link_multiply=1.1, content_adj=0.2)

  0%|          | 0/50 [00:00<?, ?it/s]

    Alpha: 115 
    Factors: 150 
    Regularisation: 550 
    Link multiplier: 1.1 
    Content adjustment: 0.2 
    Iterations: 50 
    Average rank: 26.262
    Average dg: 0.26783



(0.26783, 26.262, 115, 150, 550, 1.1, 0.2)

## Prepare test data output

**Alpha:** 115 <br>
**Number of Factors:** 150 <br>
**Regularisation:** 550 <br>
**Link multiplier:** 1.1 <br>
**Content-based score adjustment percentage:** 20%

In [908]:
alpha = 115
data = (sparse_user_item * alpha).astype('double')

# Fit the model
model = implicit.als.AlternatingLeastSquares(factors=150, regularization=550, iterations=100, random_state=0)
model.fit(data)

# Get the trained user and item vectors, converted to csr matrices
user_vecs = sparse.csr_matrix(model.user_factors)
item_vecs = sparse.csr_matrix(model.item_factors)

users, items, scores = [], [], []
num_items = 15 # For test data we only use the top 15 items for each user
for i in range(sparse_user_item.shape[0]):
    users += [i] * num_items
    recommendations = als_recommend_3(i, sparse_user_item, user_vecs, item_vecs, test, 
                                num_items, link_multiply=1.1, content_adj=0.2)
    items += recommendations[0]
    scores += recommendations[1]
    
recommendations_df = pd.DataFrame(list(zip(users, items, scores)), 
                                  columns =['user_id', 'item_id', 'scores'])

recommendations_df.drop('scores', axis=1, inplace=True)
recommendations_df.to_csv('output.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

The recommendations provide a NDCG score of 0.22894 using 50% of the data.

---

The content-based recommender system did not perform very well. The average ranking of the correct interaction from the validation set was 46.4986, which is not much better than random recommendations, which would be at around 50.5 (rankings range from 1 to 100). The average discount gain was also only 0.1148, which means that a lot of the time, the relevant item recommendation is being ranked outside the top 15, which would not register any contribution to cumulative gain in the test set as we need to recommend the top 15 from a candidate list of 100 for each user.

The image feature representations and average of the image feature representations liked by users resulted in a poor content-based recommender system. As the features were extracted from a deep learning framework, they might be more suitable in a neural collaborative filtering recommender system but for a simple similarity measure, they do not result in quality recommendations for users.

For the item-based collaborative filtering recommender, both Cosine and Jaccard similarity measures performed similarly. For the item-based method, selecting around 700 neighbours from 9,004 as similar items (for a given item in a user-item score prediction), yielded the best results. For the user-based method, around 400-500 similar users of 3,466 showed the best results against the validation set. Table 1 below summarises some comparisons between the approaches for 100, 400 and 700 neighbours.

$$
\begin{aligned}
\begin{array}{llrrrr}
{n}   & \text{Metric}                & \text{Item-based CF} &                 & \text{User-based CF} \\
      &                              & \text{Cosine}        & \text{Jaccard}  & \text{Cosine} & \text{Jaccard} \\
{100} & \text{Average rank}          & 39.5499              & 39.8266         & 56.2614       & 57.0828 \\
      & \text{Average discount gain} & 0.1715               & 0.1735          & 0.1745        & 0.1726 \\
{400} & \text{Average rank}          & 36.7764              & 36.8122         & 39.1431       & 38.8604 \\
      & \text{Average discount gain} & 0.1946               & 0.1969          & 0.1947        & 0.1959 \\
{700} & \text{Average rank}          & 36.3102              & 36.3658         & 39.1425       & 38.7106 \\
      & \text{Average discount gain} & 0.1977               & 0.1991          & 0.193         & 0.1937         
\end{array}
\end{aligned}
$$

For an optimal number of neighbours, the item-based collaborative filtering approach performs better with the Jaccard similarity measure scoring 0.1991 for average discount gain and the Cosine similarity measure scoring 0.1977. The average ranking for the item-based approach is also better than for the user-based approach.

Notably, the average rank of the user-based collaborative filtering recommender is much worse when there are too few neighbours used. Looking at the formula for the user-based approach, for a given user, the user-user similarities and proportion of items that similar users have interacted with will be the same regardless of what the item is. If none of the similar users have interacted with that item, then it will receive the same score as any other item with which none of the users have interacted with. For the item-based approach, each item will have its own set of similar items, so the ratings between items will have more variation. Under the user-based approach with 100 neighbours, the target item sometimes could not be distinguished from other items due to having too few interactions, resulting in the poorer average rank. With regard to the average discount gain, the cut-off of scores after rank 15 ensures that any rankings of the target item outside the top 15 receive equal treatment, which is why the average discount gain is unaffected for the user-based approach with 100 neighbours. Compared to the content-based approach, the collaborative filtering approaches perform markedly better, although there is still room for improvement.

The matrix factorisation models all perform much better than the content-based or collaborative filtering approaches with even a small degree of hyperparameter tuning as they generate latent features from the data that capture user-item interactions well. Finding an optimal matrix factorisation model is however very difficult, as an exhaustive search of hyperparameter permutations is computationally too expensive. A broad approach was used to identify reasonable combinations of hyperparameters, which was then used to narrow down and finetune the models.

Utilising min-max scaled content-based similarities scores was found to provide modest improvements. Scores in candidate models were increased by 15 – 20% of their counterpart content-based scores, before being rescaled. The use of adjusting the scores for photos liked by friends of friends upwards by around 10% following this, was also found to improve against the discount gain performance metric. The final model used a social link item score increase of 10% and 20% for the content-based contribution. 150 factors were used with alpha of 115 and regularisation factor of 550 to prevent overfitting with the large number of factors.

For the task of recommending photos to users on the social media platform Flickr, three types of algorithms were implemented. First, a content-based recommender was built using the item and user feature embeddings, which performed poorly. Collaborative filtering algorithms were then built from the user-item interactions data, using item-based and user-based methodologies and cosine and Jaccard similarity measures, all of which outperformed the content-based recommender. The item-based recommenders produced better average ranks of the target items than the user-based recommenders while also achieving a higher average discount gain for an optimised number of similar neighbours. Ultimately, the matrix factorisation recommenders performed much better than either of these approaches as it uses a model-based approach.